<a href="https://colab.research.google.com/github/alifarah94/DS-Unit-1-Sprint-2-Statistics/blob/master/DS_Unit_1_Sprint_Challenge_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Science Unit 1 Sprint Challenge 2

## Exploring Data, Testing Hypotheses

In this sprint challenge you will look at a dataset of people being approved or rejected for credit.

https://archive.ics.uci.edu/ml/datasets/Credit+Approval

Data Set Information: This file concerns credit card applications. All attribute names and values have been changed to meaningless symbols to protect confidentiality of the data. This dataset is interesting because there is a good mix of attributes -- continuous, nominal with small numbers of values, and nominal with larger numbers of values. There are also a few missing values.

Attribute Information:
- A1: b, a.
- A2: continuous.
- A3: continuous.
- A4: u, y, l, t.
- A5: g, p, gg.
- A6: c, d, cc, i, j, k, m, r, q, w, x, e, aa, ff.
- A7: v, h, bb, j, n, z, dd, ff, o.
- A8: continuous.
- A9: t, f.
- A10: t, f.
- A11: continuous.
- A12: t, f.
- A13: g, p, s.
- A14: continuous.
- A15: continuous.
- A16: +,- (class attribute)

Yes, most of that doesn't mean anything. A16 is a variable that indicates whether or not a person's request for credit has been approved or denied. This is a good candidate for a y variable since we might want to use the other features to predict this one. The remaining variables have been obfuscated for privacy - a challenge you may have to deal with in your data science career.

Sprint challenges are evaluated based on satisfactory completion of each part. It is suggested you work through it in order, getting each aspect reasonably working, before trying to deeply explore, iterate, or refine any given step. Once you get to the end, if you want to go back and improve things, go for it!

## Part 1 - Load and validate the data

- Load the data as a `pandas` data frame.
- Validate that it has the appropriate number of observations (you can check the raw file, and also read the dataset description from UCI).
- UCI says there should be missing data - check, and if necessary change the data so pandas recognizes it as na
- Make sure that the loaded features are of the types described above (continuous values should be treated as float), and correct as necessary

This is review, but skills that you'll use at the start of any data exploration. Further, you may have to do some investigation to figure out which file to load from - that is part of the puzzle.

Hint: If a column has the datatype of "object" even though it's made up of float or integer values, you can coerce it to act as a numeric column by using the `pd.to_numeric()` function.

In [0]:
import pandas as pd
import numpy as np
from scipy import stats
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import ttest_ind

In [0]:
# TODO

names = ['A1','A2','A3','A4','A5','A6','A7','A8','A9','A10','A11','A12','A13','A14','A15','A16']
df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/credit-screening/crx.data', header=None,names=names)

In [0]:
df.isnull().sum() # dataframe has no null values
df.describe()

,A3,A8,A11,A15
count,690.000000,690.000000,690.00000,690.000000
mean,4.758725,2.223406,2.40000,1017.385507
std,4.978163,3.346513,4.86294,5210.102598
min,0.000000,0.000000,0.00000,0.000000
25%,1.000000,0.165000,0.00000,0.000000
50%,2.750000,1.000000,0.00000,5.000000
75%,7.207500,2.625000,3.00000,395.500000
max,28.000000,28.500000,67.00000,100000.000000


In [0]:
for i in names:
  print(df[i].value_counts(), '\n')
 
"""looking throught the data i can see that there are question marks through out the dataset,
I think its ok to leave most of them the way they are but I will be replacing the question marks in A2""" 

b    468
a    210
?     12
Name: A1, dtype: int64 

?        12
22.67     9
20.42     7
22.50     6
23.58     6
25.00     6
24.50     6
19.17     6
18.83     6
20.67     6
23.08     5
23.25     5
23.00     5
33.17     5
27.67     5
27.83     5
26.17     4
26.67     4
29.50     4
22.92     4
24.75     4
41.17     4
34.17     4
25.17     4
22.08     4
35.17     4
23.50     4
32.33     4
27.25     4
25.67     4
         ..
74.83     1
17.25     1
42.17     1
17.50     1
38.42     1
32.17     1
60.08     1
46.08     1
30.00     1
36.08     1
57.83     1
73.42     1
17.83     1
48.33     1
33.92     1
36.42     1
31.00     1
30.25     1
30.08     1
49.83     1
59.50     1
39.42     1
52.33     1
54.33     1
80.25     1
30.83     1
21.42     1
30.42     1
45.17     1
29.92     1
Name: A2, Length: 350, dtype: int64 

1.500     21
0.000     19
3.000     19
2.500     19
1.250     16
0.750     16
0.500     15
5.000     14
4.000     12
1.750     12
6.500     12
2.000     10
10.000    10
1.000    

'looking throught the data i can see that there are question marks through out the dataset,\nI think its ok to leave most of them the way they are but I will be replacing the question marks in A2'

In [0]:
#Replacing ? with Nan 
df['A2']= df['A2'].replace({'?':np.NaN})
df['A14']= df['A14'].replace({'?':np.NaN})

In [0]:
# then changing the type to numeric
df['A2'] = pd.to_numeric(df['A2'])
df['A14'] = pd.to_numeric(df['A14'])

In [0]:
# all columns now should be their respected types
df.dtypes

A1      object
A2     float64
A3     float64
A4      object
A5      object
A6      object
A7      object
A8     float64
A9      object
A10     object
A11      int64
A12     object
A13     object
A14    float64
A15      int64
A16     object
dtype: object

## Part 2 - Exploring data, Testing hypotheses

The only thing we really know about this data is that A16 is the class label. Besides that, we have 6 continuous (float) features and 9 categorical features.

Explore the data: you can use whatever approach (tables, utility functions, visualizations) to get an impression of the distributions and relationships of the variables. In general, your goal is to understand how the features are different when grouped by the two class labels (`+` and `-`).

For the 6 continuous features, how are they different when split between the two class labels? Choose two features to run t-tests (again split by class label) - specifically, select one feature that is *extremely* different between the classes, and another feature that is notably less different (though perhaps still "statistically significantly" different). You may have to explore more than two features to do this.

For the categorical features, explore by creating "cross tabs" (aka [contingency tables](https://en.wikipedia.org/wiki/Contingency_table)) between them and the class label, and apply the Chi-squared test to them. [pandas.crosstab](http://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.crosstab.html) can create contingency tables, and [scipy.stats.chi2_contingency](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.chi2_contingency.html) can calculate the Chi-squared statistic for them.

There are 9 categorical features - as with the t-test, try to find one where the Chi-squared test returns an extreme result (rejecting the null that the data are independent), and one where it is less extreme.

**NOTE** - "less extreme" just means smaller test statistic/larger p-value. Even the least extreme differences may be strongly statistically significant.

Your *main* goal is the hypothesis tests, so don't spend too much time on the exploration/visualization piece. That is just a means to an end - use simple visualizations, such as boxplots or a scatter matrix (both built in to pandas), to get a feel for the overall distribution of the variables.

This is challenging, so manage your time and aim for a baseline of at least running two t-tests and two Chi-squared tests before polishing. And don't forget to answer the questions in part 3, even if your results in this part aren't what you want them to be.

In [0]:
#EXPLOR DATA
# FOR NUMERICAL USE T-TEST AND CHI SQUARE
# FOR CATAGORICAL USE CROSS TABLE 

In [0]:
# splitting data between plus and minus
plus = df[df['A16'] == "+"]
minus = df[df['A16'] == "-"]


In [0]:
plus.mean() # checking means

A2       33.720492
A3        5.904951
A8        3.427899
A11       4.605863
A14     164.421927
A15    2038.859935
dtype: float64

In [0]:
minus.mean()

A2      29.808231
A3       3.839948
A8       1.257924
A11      0.631854
A14    199.699468
A15    198.605744
dtype: float64

The minus group have lower values than the plus group in all numerical columns. 

I will check how many null values eact group has

In [0]:
plus.isnull().sum()

A1     0
A2     2
A3     0
A4     0
A5     0
A6     0
A7     0
A8     0
A9     0
A10    0
A11    0
A12    0
A13    0
A14    6
A15    0
A16    0
dtype: int64

In [0]:
minus.isnull().sum()
# it seems that there is not much of a diffrence overall when it comes to missing values

A1      0
A2     10
A3      0
A4      0
A5      0
A6      0
A7      0
A8      0
A9      0
A10     0
A11     0
A12     0
A13     0
A14     7
A15     0
A16     0
dtype: int64

In [0]:
# A2 T-TEST

ttest_ind(plus['A2'], minus['A2'],
          nan_policy='omit')


Ttest_indResult(statistic=4.2922156166315535, pvalue=2.0276637071781407e-05)

In [0]:
# A15 T-TEST

ttest_ind(plus['A15'], minus['A15'],
          nan_policy='omit')

Ttest_indResult(statistic=4.680216020964486, pvalue=3.4520256956287944e-06)

In [0]:
# function to do a ttest for each numerical cloumn

def ttest_tester(x,y):
  num_com = ['A2','A3','A8','A11','A14','A15']
  for i in num_com:
    print(ttest_ind(x[i], y[i],
          nan_policy='omit'))
    

In [0]:
ttest_tester(plus, minus)

Ttest_indResult(statistic=4.2922156166315535, pvalue=2.0276637071781407e-05)
Ttest_indResult(statistic=5.52998337614816, pvalue=4.551680702308068e-08)
Ttest_indResult(statistic=8.935819983773698, pvalue=3.6710537401601785e-18)
Ttest_indResult(statistic=11.667004222431277, pvalue=7.957718568079967e-29)
Ttest_indResult(statistic=-2.6358251986645476, pvalue=0.008586135473979569)
Ttest_indResult(statistic=4.680216020964486, pvalue=3.4520256956287944e-06)


In [0]:
# A9 CROSS-TABLE

A9_cross_tab =pd.crosstab(df['A16'], df['A9'])
A9_cross_tab

A9,f,t
A16,,
+,23,284
-,306,77


In [0]:
# A9 CHI TEST
chi_squared_A9,pvalue_A9,dof_A9,expected_A9 = stats.chi2_contingency(A9_cross_tab)
print(chi_squared_A9)
print(pvalue_A9)
print(dof_A9)
print(expected_A9)

355.2038167412799
3.1185900878457007e-79
1
[[146.38115942 160.61884058]
 [182.61884058 200.38115942]]


In [0]:
# A9 CROSS TABLE
A10_cross_tab =pd.crosstab(df['A16'], df['A10'])
A10_cross_tab

A10,f,t
A16,,
+,98,209
-,297,86


In [0]:
# A10 CHI TEST

chi_squared_A10,pvalue_A10,dof_A10,expected_A10 = stats.chi2_contingency(A10_cross_tab)
print(chi_squared_A10)
print(pvalue_A10)
print(dof_A10)
print(expected_A10)

143.06956205083145
5.675727374527571e-33
1
[[175.74637681 131.25362319]
 [219.25362319 163.74637681]]


## Part 3 - Analysis and Interpretation

Now that you've looked at the data, answer the following questions:

- Interpret and explain the two t-tests you ran - what do they tell you about the relationships between the continuous features you selected and the class labels?
- Interpret and explain the two Chi-squared tests you ran - what do they tell you about the relationships between the categorical features you selected and the class labels?
- What was the most challenging part of this sprint challenge?

Answer with text, but feel free to intersperse example code/results or refer to it from earlier.



# T-test
the Ttest for A2 showed that the graph skewed a little bit to the right and had a low p-value meaning that the result wasnt random. the ttest came out as (+4) showing that there wasnt much diffrence in both groups mean for A2. The plus group had a higher mean than the minus group but only slightly and the minus group had more null values, here we can just about reject the null Hypothesis that both groups would be the same because. If the minus group had less missing values maybe the null  Hypothesis would be right.


The ttest for A15 was suprising to me. I picked the A15 because that's where both groups means differed the most. The test came out with the same results as the first ttest, with +4  as the ttest result and <0.05 as the pvalue. What I took from this result was the that there was not much diffrence between each groups mean but looking at both thier means sepratly I see a big diffrence so...



# Chi squared
I decided to do a chi squared test on 2 columns that had the same the same characters too see if they would have similar scores. 

The A9 chi square test came out high at 355 showing that there is no relationship between the groups (f/t)

similarly with the A10 chi square test; it also had a high score but not much so as A9



# Challenge
I think the hardest aspect was looking at the data and deciding what to do with it. How much work I should put in, its very easy to get lost.

